In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

class pricing_formula:
    def __init__(self, variables):
        self.S0, self.K, self.r, self.q, self.sigma, self.T, self.simu_num, self.rep, self.n = variables
        self.rep = int(self.rep)
        self.simu_num = int(self.simu_num)
        self.n = int(self.n)

    def BS_model(self):
        self.d1 = ( np.log(self.S0/self.K) + (self.r-self.q+(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T))
        self.d2 = ( np.log(self.S0/self.K) + (self.r-self.q-(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T))
        self.Nd1 = norm.cdf(self.d1)
        self.Nd2 = norm.cdf(self.d2)
        c = self.S0 * np.exp(-self.q * self.T) * self.Nd1 - self.K * np.exp( -self.r * self.T) * self.Nd2
        # put option price by put-call parity
        p = c + np.exp(-self.r * self.T) * self.K - self.S0 * np.exp(-self.q * self.T)
        # put option price by formula
        # p = self.K * np.exp(-self.r*self.T)*(1-self.Nd2) - self.S0 * np.exp(-self.q * self.T) * (1 - self.Nd1)
        return (c,p)

    def BS_info(self):
        print("d1 = ", self.d1, ", d2 = ", self.d2)
        print("N(d1) = ", self.Nd1, ", N(d2) = ", self.Nd2)

    def Monte_Carlo(self):
        self.mean = np.log(self.S0) + (self.r-self.q-(self.sigma**2)/2) * self.T
        self.std = self.sigma * np.sqrt(self.T)

        #print(self.mean, self.std)
        self.mean_array_c = np.zeros(self.rep)
        self.mean_array_p = np.zeros(self.rep)
        for i in range(self.rep):
            sample = np.random.normal(self.mean, self.std, self.simu_num)
            sample = np.exp(sample)
            df = pd.DataFrame(sample)
            df.columns = ['ST']
            df['ST-K'] = df['ST'] - self.K
            df['zero'] = 0
            df['K-ST'] = self.K - df['ST']
            df['Payoff_C'] = df[['ST-K','zero']].max(axis=1)
            df['Payoff_P'] = df[['K-ST','zero']].max(axis=1)
            df['PV_C'] = df['Payoff_C'] * np.exp(-self.r * self.T)
            df['PV_P'] = df['Payoff_P'] * np.exp(-self.r * self.T)
            mean_c = df['PV_C'].mean()
            mean_p = df['PV_P'].mean()
            self.mean_array_c[i] = mean_c
            self.mean_array_p[i] = mean_p

        call_mean = self.mean_array_c.mean()
        call_std = self.mean_array_c.std()
        put_mean = self.mean_array_p.mean()
        put_std = self.mean_array_p.std()
        return  call_mean-2*call_std,  call_mean+2*call_std, put_mean-2*put_std,  put_mean+2*put_std

    def CRR_binomial(self):
        self.delta_t = self.T/self.n
        self.u = np.exp(self.sigma * np.sqrt(self.delta_t))
        self.d = 1/self.u
        self.p = ( np.exp((self.r - self.q) * self.delta_t) - self.d) / (self.u - self.d)
        #print("u, d, p = ", self.u,self.d,self.p)
        self.S_array = []
        EC_array = []
        EP_array = []
        AC_array = []
        AP_array = []
        for i in range(self.n+1):
            self.S_array.append([])
            EC_array.append([])
            EP_array.append([])
            AC_array.append([])
            AP_array.append([])
            #print("i = ", i)
            for j in range(i+1):
                #print("j = ", j)
                stock_price = self.S0 * (self.u**(i-j)) * (self.d**j)
                self.S_array[i].append(stock_price)
                EC_array[i].append(-1)
                EP_array[i].append(-1)
                AC_array[i].append(-1)
                AP_array[i].append(-1)

        for j in range(self.n+1):
            EC_array[self.n][j] = np.max([self.S_array[self.n][j]-self.K,0])
            EP_array[self.n][j] = np.max([self.K - self.S_array[self.n][j],0])
            AC_array[self.n][j] = np.max([self.S_array[self.n][j]-self.K,0])
            AP_array[self.n][j] = np.max([self.K - self.S_array[self.n][j],0])

        #print(len(S_array))
        #print(len(C_array))
        for i in range(self.n-1,-1,-1):
            for j in range(i+1):
                #print("[i,j] = [%d,%d]" % (i,j))
                EC_array[i][j] = np.exp(-self.r * self.delta_t) * ((self.p * EC_array[i+1][j]) + (1-self.p)*(EC_array[i+1][j+1]))
                EP_array[i][j] = np.exp(-self.r * self.delta_t) * ((self.p * EP_array[i+1][j]) + (1-self.p)*(EP_array[i+1][j+1]))
                AC_array[i][j] = np.max([np.exp(-self.r * self.delta_t) * ((self.p * AC_array[i+1][j]) + (1-self.p)*(AC_array[i+1][j+1])),\
                                         self.S_array[i][j]-self.K])
                AP_array[i][j] = np.max([np.exp(-self.r * self.delta_t) * ((self.p * AP_array[i+1][j]) + (1-self.p)*(AP_array[i+1][j+1])),\
                                         self.K - self.S_array[i][j]])

        #print("Stock price = \n", self.S_array)
        #print("EC array = \n", EC_array)
        #print("EP array = \n", EP_array)
        #print("AC array = \n", AC_array)
        #print("AP array = \n", AP_array)
        return EC_array[0][0], EP_array[0][0], AC_array[0][0], AP_array[0][0]

    def CRR_binomial_one_column(self):
        # Copy S array for payoff arrays
        EC_array = self.S_array[self.n].copy()
        EP_array = self.S_array[self.n].copy()
        AC_array = self.S_array[self.n].copy()
        AP_array = self.S_array[self.n].copy()

        # Transfer arrays into terminal payoff arrays
        for j in range(self.n+1):
            EC_array[j] = np.max([EC_array[j]-self.K,0])
            EP_array[j] = np.max([self.K - EP_array[j],0])
            AC_array[j] = np.max([AC_array[j]-self.K,0])
            AP_array[j] = np.max([self.K - AP_array[j],0])

        # Do backward induction
        for i in range(self.n-1,-1,-1):
            for j in range(i+1):
                EC_array[j] = np.exp(-self.r * self.delta_t) * ((self.p * EC_array[j]) + (1-self.p)*(EC_array[j+1]))
                EP_array[j] = np.exp(-self.r * self.delta_t) * ((self.p * EP_array[j]) + (1-self.p)*(EP_array[j+1]))
                AC_array[j] = np.max([np.exp(-self.r * self.delta_t) * ((self.p * AC_array[j]) + (1-self.p)*(AC_array[j+1])),\
                                      (self.S0 * self.u ** (i-j) * self.d ** j) - self.K])
                AP_array[j] = np.max([np.exp(-self.r * self.delta_t) * ((self.p * AP_array[j]) + (1-self.p)*(AP_array[j+1])),\
                                      self.K - (self.S0 * self.u ** (i-j) * self.d ** j)])
        return EC_array[0], EP_array[0], AC_array[0], AP_array[0]

    def combinatorial_method(self):
        self.delta_t = self.T/self.n
        self.u = np.exp(self.sigma * np.sqrt(self.delta_t))
        self.d = 1/self.u
        self.p = ( np.exp((self.r - self.q) * self.delta_t) - self.d) / (self.u - self.d)
        c_payoff = 0
        p_payoff = 0
        for j in range(self.n+1):
            ln_sum = 0
            for k in range(1, self.n+1):
                ln_sum += np.log(k)
            for k in range(1, j+1):
                ln_sum -= np.log(k)
            for k in range(1, self.n-j+1):
                ln_sum -= np.log(k)
            ln_sum += ( (self.n-j)*np.log(self.p) + j*np.log(1-self.p))
            c_payoff += np.exp(ln_sum) * np.max([self.S0*(self.u**(self.n-j))*(self.d**j)-self.K,0])
            p_payoff += np.exp(ln_sum) * np.max([self.K - self.S0*(self.u**(self.n-j))*(self.d**j),0])
        final_c = np.exp(-self.r*self.T) * c_payoff
        final_p = np.exp(-self.r*self.T) * p_payoff
        return final_c, final_p

In [ ]:
order = ['S0', 'K', 'r', 'q', 'sigma', 'T', 'number of simulations', 'number of repetitions','n']
var = []

for i in range(len(order)):
    var.append(float(input("Please type " + order[i] + ":")))

print()
model = pricing_formula(var)

price = model.BS_model()
print("1. BS Model(european options): Call price = %f; put price = %f\n"%price)

CI = model.Monte_Carlo()
print("2. Monte Carlo Simulation(european options):\n  Confidence interval for call option = [%f, %f]\n  Confidence interval for put option = [%f, %f]\n" % CI)

eu_c, eu_p, am_c, am_p = model.CRR_binomial()
print("3. CRR Binomial Tree: ")
print("European call price = %f, european put price = %f" % (eu_c, eu_p))
print("American call price = %f, american put price = %f\n" % (am_c, am_p))

# Bonus 1
eu_c, eu_p, am_c, am_p = model.CRR_binomial_one_column()
print("Bonus 1. CRR Binamial Tree using one column vector: ")
print("European call price = %f, european put price = %f" % (eu_c, eu_p))
print("American call price = %f, american put price = %f\n" % (am_c, am_p))

# Bonus 2
price = model.combinatorial_method()
print("Bonus 2. Combinatorial method: ")
print("European call price = %f, european put price = %f\n" % price)


Please type S0: 50
Please type K: 56.115
Please type r: 0.1
Please type q: 0
Please type sigma: 0.4
Please type T: 0.25
Please type number of simulations: 10000
Please type number of repetitions: 20
Please type n: 3



1. BS Model(european options): Call price = 2.223446; put price = 6.952961

2. Monte Carlo Simulation(european options):
  Confidence interval for call option = [2.112608, 2.341153]
  Confidence interval for put option = [6.783586, 7.165811]

3. CRR Binomial Tree: 
European call price = 1.859108, european put price = 6.588623
American call price = 1.859108, american put price = 7.155016

Bonus 1. CRR Binamial Tree using one column vector: 
European call price = 1.859108, european put price = 6.588623
American call price = 1.859108, american put price = 7.155016

Bonus 2. Combinatorial method: 
European call price = 1.859108, european put price = 6.588623



In [ ]:
order = ['S0', 'K', 'r', 'q', 'sigma', 'T', 'number of simulations', 'number of repetitions','n']
var = []

for i in range(len(order)):
    var.append(float(input("Please type " + order[i] + ":")))

print()
model = pricing_formula(var)

price = model.BS_model()
print("1. BS Model(european options): Call price = %f; put price = %f\n"%price)

CI = model.Monte_Carlo()
print("2. Monte Carlo Simulation(european options):\n  Confidence interval for call option = [%f, %f]\n  Confidence interval for put option = [%f, %f]\n" % CI)

eu_c, eu_p, am_c, am_p = model.CRR_binomial()
print("3. CRR Binomial Tree: ")
print("European call price = %f, european put price = %f" % (eu_c, eu_p))
print("American call price = %f, american put price = %f\n" % (am_c, am_p))

# Bonus 1
eu_c, eu_p, am_c, am_p = model.CRR_binomial_one_column()
print("Bonus 1. CRR Binamial Tree using one column vector: ")
print("European call price = %f, european put price = %f" % (eu_c, eu_p))
print("American call price = %f, american put price = %f\n" % (am_c, am_p))

# Bonus 2
price = model.combinatorial_method()
print("Bonus 2. Combinatorial method: ")
print("European call price = %f, european put price = %f\n" % price)

In [ ]:
order = ['S0', 'K', 'r', 'q', 'sigma', 'T', 'number of simulations', 'number of repetitions','n']
var = []

for i in range(len(order)):
    var.append(float(input("Please type " + order[i] + ":")))

print()
model = pricing_formula(var)

# Bonus 2
price = model.combinatorial_method()
print("Bonus 2. Combinatorial method: ")
print("European call price = %f, european put price = %f\n" % price)

Please type S0: 50
Please type K: 50
Please type r: 0.1
Please type q: 0.05
Please type sigma: 0.4
Please type T: 0.5
Please type number of simulations: 10000
Please type number of repetitions: 20
Please type n: 10000



Bonus 2. Combinatorial method: 
European call price = 6.039485, european put price = 4.835461

